# Activate the cells via Ctrl+Enter or "Run" button

In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
from datetime import datetime, timedelta, date
import datetime
import vertica_python
import io
from tqdm import tqdm
from clickhouse_driver import Client
import xlsxwriter

login = 
password = 

#connect to Clickhouse

client = Client('',
               user = login,
               password = password,
               database = 'tracker')

client_1 = Client('',
               user = login,
               password = password,
               database = 'search_analytics')

settings = {'max_rows_to_group_by':1000000000000000000000000,
            'max_execution_time':9223372,
            'max_result_rows' :1000000000000000000000000
           }

conn_info_my = {
    'host': '',
    'port': 5433,
    'user': login,
    'password': password,
    'database': 'OLAP'}

conn = vertica_python.connect(**conn_info_my)

# Rule. You need to specify sku, brand id, category id for which you will receive data at the end of the script
# Restrictions:
### Sku: To 100
### Brand id: Just 1
### Cat id: To 10 


In [2]:
date_start = '2023-02-10' # Enter date start
date_end = '2023-02-10' # Enter date end

sku = (154444481, 149250290, 563429549, 149250291, 204273807, 204273814, 149250284, 20069121, 163263828, 166508186, 185453932, 531754347, 20069120, 20069124, 166508187) # Вводим sku через запятую
brand_id = [7115299] # Enter Brand id with a comma
cat_id = (7058, 30749, 7064, 30752) # Enter Categories id with a comma


start = datetime.datetime.strptime(date_start, '%Y-%m-%d') 
end = datetime.datetime.strptime(date_end, '%Y-%m-%d') 

daterange = [(start + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(((end-start).days)+1)]

print(f'Period: {date_start} - {date_end}')
print(f'SKU: {sku}')
print(f'Brand id: {brand_id}')
print(f'Categories: {cat_id}')

Period: 2023-02-10 - 2023-02-10
SKU: (154444481, 149250290, 563429549, 149250291, 204273807, 204273814, 149250284, 20069121, 163263828, 166508186, 185453932, 531754347, 20069120, 20069124, 166508187)
Brand id: [7115299]
Categories: (7058, 30749, 7064, 30752)


# Start unloading data step by step

# Download reviews on SKU


In [3]:
reviews = pd.DataFrame()    
for date_1 in daterange:
    print(f'{date_1} - Uploaded')
    reviewss = f"""SELECT DISTINCT date,
    page_sku            AS sku,
    MAX (properties_quantity) AS quantity,
    AVG (properties_rating)   AS rating
    FROM tracker.events
    WHERE attributes_namespace = 'bx'
    AND action_widget IN ('rpProduct.ugcCounters', 'rpProduct.webReviewProductScore')
    AND date = '{date_1}'
    AND page_sku IN {sku}
    AND properties_rating IS NOT NULL 
    AND properties_rating != 0
    GROUP BY date, page_sku""" 
    result = pd.DataFrame(client.execute(reviewss, settings=settings)).rename(columns={0:'date',1:'sku',2:'reviews_qty',3:'rating'})
    reviews = pd.concat([result, reviews])
print('All dates uploaded. Check dataset')
reviews.head(5)

2023-02-10 - Uploaded
All dates uploaded. Check dataset


,date,sku,reviews_qty,rating
0,2023-02-10,204273807,16019,4.813658
1,2023-02-10,149250290,36624,4.807851
2,2023-02-10,20069124,12712,4.802891
3,2023-02-10,149250291,36624,4.807594
4,2023-02-10,149250284,26423,4.801653


# Download key queries by category

In [4]:
cat_query = pd.DataFrame()    

for date_2 in daterange:
    print(f'{date_2} - Uploaded')
    cat_queryy = f"""SELECT event_date as date,
           concat(concat(concat(navigation_category_name_1, ' - ',
                                navigation_category_name_2), ' - ',
                         navigation_category_name_3), ' - ',
                  navigation_category_name_4) AS categories,
           query,
           sum(search)         AS searches,
           ROUND(MAX(mean_bid)*1000, 2) AS mean_bid,
           SUM(gmv) AS gmv,
           SUM(ord) AS orders,
           SUM(ord_users) AS clients
    FROM (SELECT event_date, query, cat_predict, search, mean_bid, gmv, ord, ord_users
          FROM search_analytics.wordstat_aggregates_v2
          WHERE 1 = 1
            AND event_date = '{date_2}'
            AND cat_predict IN {cat_id}
                ) AS T1
             JOIN
         (SELECT id,
                 navigation_category_name_1,
                 navigation_category_name_2,
                 navigation_category_name_3,
                 navigation_category_name_4
          FROM dictionary.navigation_category
          WHERE id IN {cat_id}
                    ) AS T2 ON toUInt64(T1.cat_predict) = T2.id
    GROUP BY date,
             concat(concat(concat(navigation_category_name_1, ' - ',
                                  navigation_category_name_2), ' - ',
                           navigation_category_name_3), ' - ',
                    navigation_category_name_4),
             query
    ORDER BY date,
             categories, 
             searches DESC"""
    result = pd.DataFrame(client_1.execute(cat_queryy, settings=settings)).rename(columns={0:'date',1:'categories',2:'query',3:'searches',4:'mean_bid',5:'gmv',6:'orders',7:'clients'})
    cat_query = pd.concat([result, cat_query])
print('All dates uploaded. Check dataset')
cat_query.sort_values('date').head(5)

2023-02-10 - Uploaded
All dates uploaded. Check dataset


,date,categories,query,searches,mean_bid,gmv,orders,clients
0,2023-02-10,Детские товары - Подгузники и гигиена - -,pampers,439,250.00,48652,21,18
5871,2023-02-10,Детские товары - Подгузники и гигиена - Подгуз...,джунис роял флаффи,2,0.00,0,0,0
5870,2023-02-10,Детские товары - Подгузники и гигиена - Подгуз...,подгузники трусики беби гоу 4,2,74.38,0,0,0
5869,2023-02-10,Детские товары - Подгузники и гигиена - Подгуз...,мимисо подгузники 4,2,71.18,0,0,0
5868,2023-02-10,Детские товары - Подгузники и гигиена - Подгуз...,памперсы йокосан l,2,89.98,0,0,0


# Download Key Queries by Brand id

In [5]:
queries_by_brandid = pd.DataFrame()
for date_5 in daterange:
    print(f'{date_5} - Uploaded')
    queries_by_brandidd = f"""
            SELECT  event_date AS date,
                query,
                brand_id,
                b.name AS brand_name,
                SUM (total_gmv) AS gmv
        FROM search_analytics.query_top_brand a
        JOIN dictionary.brand b ON a.brand_id = b.id
        WHERE brand_id IN {brand_id}
        AND event_date = '{date_5}'
        GROUP BY event_date, 
                 query,
                 brand_id,
                 b.name
        ORDER BY gmv DESC"""
    result = pd.DataFrame(client_1.execute(queries_by_brandidd, settings=settings)).rename(columns={0:'date',1:'query',2:'brand_id',3:'brand_name',4:'gmv'})
    queries_by_brandid = pd.concat([result, queries_by_brandid])
print('All dates uploaded. Check dataset')
queries_by_brandid.sort_values('gmv', ascending=False).head(5)

2023-02-10 - Uploaded
All dates uploaded. Check dataset


,date,query,brand_id,brand_name,gmv
0,2023-02-10,huggies elite soft 3,7115299,Huggies,52304
1,2023-02-10,подгузники трусики,7115299,Huggies,46760
2,2023-02-10,подгузники,7115299,Huggies,33310
3,2023-02-10,huggies elite soft 4,7115299,Huggies,31114
4,2023-02-10,huggies elite soft,7115299,Huggies,30602


# Unloading basic metrics by SKU

In [6]:
metrics = pd.DataFrame()    

for date_3 in daterange:
    print(f'{date_3} - Uploaded')
    metricss = f"""SELECT  a.FactDate AS date,
            a.RezonItemID AS sku,
            COUNT (DISTINCT RezonClientID) AS clients,
            COUNT (RezonClientOrderID) AS orders,
            SUM (SalesCommercial * Qty) AS gmv,
            SUM (Qty) AS qty,
            AVG (SalePrice) AS avg_sale_price,
            SUM (SalesCommercial*Qty) / COUNT (RezonClientID) AS arppu,
            COUNT (RezonClientOrderID) / COUNT (DISTINCT RezonClientID) AS freq
    FROM metazonbeeeye.action a
             left join dwh_data.Anc_Item anc on anc.SourceKey = a.RezonItemID
             left join dwh_data.Atr_Item_Name i on i.ItemId = anc.ItemId
             left join dwh.Fact_Item_Brand fb ON anc.ItemId = fb.ItemID
             left join dwh.Dim_Brand br on br.BrandId = fb.BrandId
    WHERE FactDate = '{date_3}'
    AND ActionTypeID = 101
    AND a.RezonItemID IN {sku}
    GROUP BY 	a.FactDate,
                a.RezonItemID
    ORDER BY FactDate DESC"""
    vertica_query = pd.read_sql(metricss, conn)  
    metrics = pd.concat([vertica_query, metrics])
print('All dates uploaded. Check dataset')
metrics.head(5)

2023-02-10 - Uploaded
All dates uploaded. Check dataset


,date,sku,clients,orders,gmv,qty,avg_sale_price,arppu,freq
0,2023-02-10,20069120,50,51,69708.9103,51,1499.088431,1366.841378,1.020000
1,2023-02-10,20069121,71,77,106963.0927,78,1508.706494,1389.131074,1.084507
2,2023-02-10,20069124,64,69,94302.3650,69,1497.691014,1366.700942,1.078125
3,2023-02-10,149250284,67,68,77271.4284,69,1230.051029,1136.344535,1.014925
4,2023-02-10,149250290,53,54,119650.3264,55,2391.043704,2215.746785,1.018868


# Unload cross-purchase categories

In [7]:
cross_cat = pd.DataFrame()

for my_date in daterange:
    print(f"{my_date} - Uploaded")
    query_cross = f"""with orders as
        (
            select distinct a.RezonClientOrderID
            from metazonbeeeye.action a
                join dwh_data.anc_item ai on a.rezonitemid=ai.SourceKey
                join dwh.Fact_Item_Brand as fib on ai.ItemId = fib.ItemId
                join dwh.Dim_Brand db on db.BrandID = fib.BrandID
            where a.ActionTypeID = 101
                 and db.ExternalKey IN ({brand_id[0]})
                and a.FactDate = '{my_date}'
        )
        select dnc1.Name as cross_cat,
            anc.SourceKey as cross_cat_id,
            dnc.Level AS cat_lvl,
            count(distinct a.RezonClientOrderID) as orders_together
        from metazonbeeeye.action a
            join dwh_data.anc_item ai on a.rezonitemid=ai.SourceKey
            join dwh.Fact_Item_Brand as fib on ai.ItemId = fib.ItemId
            join dwh.Dim_Brand db on db.BrandID = fib.BrandID
            join dwh.Fact_Item_NavigationalCategory finc on ai.itemid = finc.itemid
            join dwh.Dim_NavigationalCategory dnc on finc.NavigationalCategoryId = dnc.NavigationalCategoryId
            join dwh.Dim_NavigationalCategory dnc1 on dnc.Path[1] = dnc1.NavigationalCategoryId
            join dwh_data.Anc_NavigationalCategory anc on anc.NavigationalCategoryId = dnc1.Path[1]
        where a.ActionTypeID = 101
             and db.ExternalKey != ({brand_id[0]})
            and a.RezonClientOrderID in (select * from orders)
        group by dnc1.Name, anc.SourceKey, dnc.Level
        order by Orders desc"""
    vertica_query_cross = pd.read_sql(query_cross, conn)  
    cross_cat = pd.concat([vertica_query_cross, cross_cat])
print('All dates uploaded. Check dataset')
cross_cat['core_brand'] = queries_by_brandid.brand_name.iloc[0]
cross_cat['core_brand_id'] = brand_id[0]
cross_cat=cross_cat[['core_brand', 'core_brand_id', 'CatName2', 'CatID2', 'cat_lvl', 'orders']]
cross_cat.head(5)

2023-02-10 - Uploaded
All dates uploaded. Check dataset


,core_brand,core_brand_id,CatName2,CatID2,cat_lvl,orders
0,Huggies,7115299,Другие товары,2501000,8,430
1,Huggies,7115299,Товары для мам и детей,7000000,5,423
2,Huggies,7115299,Услуги,38022,3,409
3,Huggies,7115299,Подгузники и гигиена,7058,4,347
4,Huggies,7115299,Личная гигиена,6310,4,165


# Unloading views and weighted average SKU position

In [8]:
sku_query = pd.DataFrame()    
for date_4 in daterange:    
    print(f'{date_4} - Uploaded')
    sku_queryy = f"""WITH cte AS
    (
    select date,
        object_sku,
        properties_number,
        countIf(action_type='view') views
    from tracker.events
    where date = '{date_4}'
        and attributes_namespace = 'bx'
        and action_type in ('view', 'click', 'to_cart')
        and action_widget = 'catalog.searchResultsV2'
        AND object_sku IN {sku}
    group by date, object_sku, properties_number
    order by date
    )
    SELECT 	date,
            object_sku AS sku,
            SUM(views),
            SUM(properties_number * views)/ SUM(views) AS weightAvg
    FROM cte
    GROUP BY date, object_sku"""  
    result = pd.DataFrame(client.execute(sku_queryy, settings=settings)).rename(columns={0:'date',1:'sku',2:'views',3:'weightAvg'})
    sku_query = pd.concat([result, sku_query])
print('All dates uploaded. Check dataset')
sku_query.head(5)

2023-02-10 - Uploaded
All dates uploaded. Check dataset


,date,sku,views,weightAvg
0,2023-02-10,204273807,9482,18.384096
1,2023-02-10,149250290,8892,24.834683
2,2023-02-10,20069124,3887,39.888603
3,2023-02-10,149250291,3615,42.439557
4,2023-02-10,149250284,8799,25.235368


# Unload a number of additions to the cart and favorites

In [9]:
a2c_df = pd.DataFrame()

for date_6 in daterange:
    print(f"{date_6} - Uploaded")
    query_a2c = f"""select date,
                object_sku AS sku,
                countIf(uuid, (object_type = 'product' and action_type = 'to_cart')) product_a2c,
                countIf(uuid, (object_type = 'product' and action_type = 'favorite')) product_favorite
        from tracker.events
        where date = '{date_6}'
            and object_sku IN {sku}
            and attributes_namespace = 'bx'
            and attributes_platform != ''
            and action_type in ('click', 'to_cart', 'favorite', 'page_view')
            and page_current = 'pdp'
        group by date, object_sku"""
    result = pd.DataFrame(client.execute(query_a2c, settings=settings)).rename(columns={0:'date',1:'sku',2:'a2c',3:'a2f',})
    a2c_df = pd.concat([result, a2c_df])
a2c_df.head(5)

2023-02-10 - Uploaded


,date,sku,a2c,a2f
0,2023-02-10,204273807,145,13
1,2023-02-10,149250290,35,4
2,2023-02-10,20069124,36,8
3,2023-02-10,149250291,29,3
4,2023-02-10,149250284,37,7


# This is the last step. Download all our data to Excel file. It will be downloaded here: https://jupyter001.s.o3.ru/user/

In [10]:
table_total = reviews.merge(sku_query)
table_total_1 = table_total.merge(metrics)
table_total_2 = table_total_1.merge(a2c_df)


writer = pd.ExcelWriter(f'Brand_audit_{queries_by_brandid.brand_name.iloc[0]}_{date_start} - {date_end}.xlsx', engine = 'xlsxwriter')
table_total_1.to_excel(writer, sheet_name='Report_by_sku')
cat_query.to_excel(writer, sheet_name='Report_by_cat')
queries_by_brandid.to_excel(writer, sheet_name='Report_by_brandId')
cross_cat.to_excel(writer, sheet_name='Report_cross_categories')
writer.close()

print('Dataset uploaded')
print(f'Dates: {date_start} - {date_end}')
print('Brand: ', queries_by_brandid.brand_name.iloc[0])

Dataset uploaded
Dates: 2023-02-10 - 2023-02-10
Brand:  Huggies
